In [1]:
import _init_paths
from utils import *
from BoundingBox import BoundingBox
from BoundingBoxes import BoundingBoxes
from Evaluator import *
from pathlib import Path
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Pre Processing sequence dict

In [2]:
pre_processing_seq_dict = {
    "seq_0" : [], # for raw seq
    "seq_1" : ["gray"],
    "seq_2" : ["hsv"],
    "seq_3" : ["sharpen"],
    "seq_4" : ["gray", "bilateral_blur", "threshold_mean"],
    "seq_5" : ["gray", "bilateral_blur", "threshold_gaussian"],
    "seq_6" : ["gray", "bilateral_blur", "threshold_otsu"],
    "seq_7" : ["median_blur"],
    "seq_8" : ["gaussian_blur"],
    "seq_9" : ["bilateral_blur"],
    "seq_10" : ["fastnl_blur"],
    "seq_11" : ["gray", "bilateral_blur", "threshold_otsu", "opening"],
    "seq_12" : ["gray", "bilateral_blur", "threshold_otsu", "closing"],
    "seq_13" : ["opening"],
    "seq_14" : ["closing"],
    "seq_15" : ["gray", "sobel"],
    "seq_16" : ["gray", "laplacian"],
    "seq_17" : ["gray", "canny"]
}

def get_seq_name(seq_id):
    if seq_id not in pre_processing_seq_dict.keys():
        return "not found for dataset id: " + seq_id
    return " > ".join(get_seq_operations(seq_id))

def get_seq_operations(seq_id):
    return pre_processing_seq_dict[seq_id]

#### Change below line to evaluate other models results

In [4]:
ground_truth_path = Path("../../../dataset/object_detection/annotations/annotations.csv")
images_path = Path("../../../dataset/object_detection/images")
model_results_path = Path("../experiment_results/model_results.csv")

### Test the evaluation algorithms

In [5]:
ground_truth_df = pd.read_csv(ground_truth_path)

In [6]:
# detections_df = detections_df.sort_values(["image_name", "detection_score"])
# detections_df.to_csv(detections_path, index=False)

### Load Custom dataset Labels

In [7]:
custom_dataset_labels = list(set(ground_truth_df["label"]))

#### Boxes for calculating precision

In [8]:
def get_bboxes(detections_df_per_seq_id):
    boxes = BoundingBoxes()
    # first create a grnd truth boxes using ground_truth_df
    for index, row in ground_truth_df.iterrows():
        box = BoundingBox(imageName = row["image_name"], classId=row["label"], x=row["xmin"], y=row["ymax"],
                          w=(row["xmax"]-row["xmin"]), h=(row["ymax"]-row["ymin"]), typeCoordinates=CoordinatesType.Absolute,
                             bbType=BBType.GroundTruth, format=BBFormat.XYWH)
        boxes.addBoundingBox(box)
        
    # then create a detection boxes using detections_df_per_seq_id
    for index, row in detections_df_per_seq_id.iterrows():
        box = BoundingBox(imageName = row["image_name"], classId=row["label"], classConfidence= row["detection_score"],
                          x=row["xmin"], y=row["ymax"], w=(row["xmax"]-row["xmin"]), h=(row["ymax"]-row["ymin"]), 
                          typeCoordinates=CoordinatesType.Absolute,
                          bbType=BBType.Detected, format=BBFormat.XYWH)
        boxes.addBoundingBox(box)
        
    return boxes

#### Boxes for drawing box on images

In [9]:
# def get_bboxes_draw(detections_df_per_seq_id):
#     boxes = BoundingBoxes()
#     # first create a grnd truth boxes using test_grnd
#     for index, row in ground_truth_df.iterrows():
#         box = BoundingBox(imageName = row["image_name"], classId=row["label"], x=row["xmin"], y=row["ymax"],
#                           w=row["xmax"], h=row["ymin"], typeCoordinates=CoordinatesType.Absolute,
#                              bbType=BBType.GroundTruth, format=BBFormat.XYX2Y2)
#         boxes_draw.addBoundingBox(box)
        
#     # then create a detection boxes using test_det
#     for index, row in detections_df.iterrows():
#         box = BoundingBox(imageName = row["image_name"], classId=row["label"], classConfidence= row["detection_score"],
#                           x=row["xmin"], y=row["ymax"], w=row["xmax"], h=row["ymin"], 
#                           typeCoordinates=CoordinatesType.Absolute,
#                           bbType=BBType.Detected, format=BBFormat.XYX2Y2)
#         boxes_draw.addBoundingBox(box)
        
#     return boxes

In [10]:
def get_metrics_per_label(boxes):
    evaluator = Evaluator()
    metricsPerClass = evaluator.GetPascalVOCMetrics(boxes, IOUThreshold=0.5)
    return metricsPerClass

In [11]:
model_results_columns = ["model_name", "seq_id", "seq_name", "label", "ap"]

def store_metrics_in_csv(metricsPerClass, seq_id):
    rows = []
    for mc in metricsPerClass:
        label = mc["class"]
        if label in custom_dataset_labels:
            row = [model_name, seq_id, get_seq_name(seq_id), label, mc["AP"]]
            rows.append(row)
    
    # append mAP row for a seq_id
    rows.append([model_name, seq_id, get_seq_name(seq_id), "all", calculate_mAP(metricsPerClass)])
            
    df = pd.DataFrame(rows, columns = model_results_columns)
    model_results_df = load_model_results_csv()
    model_results_df = model_results_df.append(df)
    
    dump_model_results_df(model_results_df)

def calculate_mAP(metricsPerClass):
    ap = [mc["AP"] for mc in metricsPerClass if mc["class"] in custom_dataset_labels]
    return round(sum(ap)/len(ap), 6)
    
def load_model_results_csv():
    model_results_df = None
    # create if not exists
    if os.path.exists(model_results_path):
        model_results_df = pd.read_csv(model_results_path)
    else:
        model_results_df = pd.DataFrame(columns = model_results_columns)
        model_results_df.to_csv(model_results_path, index=False)
    return model_results_df
    
def dump_model_results_df(model_results_df):
    model_results_df.to_csv(model_results_path, header=True, index=False)

### RUN

In [12]:
models = [
#     "efficientdet_d0_coco17_tpu-32",
#     "efficientdet_d7_coco17_tpu-32",
#     "faster_rcnn_resnet50_v1_640x640_coco17_tpu-8",
     "faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8",
#     "pp-yolo",
#     "ssd_mobilenet_v2_320x320_coco17_tpu-8",
#     "ssd_resnet50_v1_fpn_640x640_coco17_tpu-8",
#     "ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8"
]

In [13]:
warnings.filterwarnings('ignore')

for model_name in models:
    detections_path = Path("../experiment_results/final_results/"+model_name+".csv")
    detections_df = pd.read_csv(detections_path)

    # note in below df, we have processing_seq_name, but it is actually a seq_id
    df_group = detections_df.groupby("processing_seq_name")
    print("generating metrics for: {0}".format(model_name))
    for seq_id in pre_processing_seq_dict.keys():
        print("reading seq_id: {0}".format(seq_id))
        per_seq_id_results = df_group.get_group(seq_id)
        boxes = get_bboxes(per_seq_id_results)
        metrics_per_label = get_metrics_per_label(boxes)
        store_metrics_in_csv(metrics_per_label, seq_id)
    print("Completed!!!")

generating metrics for: faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8
reading seq_id: seq_0
reading seq_id: seq_1
reading seq_id: seq_2
reading seq_id: seq_3
reading seq_id: seq_4
reading seq_id: seq_5
reading seq_id: seq_6
reading seq_id: seq_7
reading seq_id: seq_8
reading seq_id: seq_9
reading seq_id: seq_10
reading seq_id: seq_11
reading seq_id: seq_12
reading seq_id: seq_13
reading seq_id: seq_14
reading seq_id: seq_15
reading seq_id: seq_16
reading seq_id: seq_17
Completed!!!


### Below code is for drawing boxes on images
* uncomment it for execution

In [13]:
# import cv2
# import numpy as np
# import os
# for image_name in list(set(ground_truth_df["image_name"])):
#     im = cv2.imread('../../../dataset/object_detection/images/'+image_name)
#     # Add bounding boxes
#     im = boxes_draw.drawAllBoundingBoxes(im, image_name)
#     # Uncomment the lines below if you want to show the images
#     #cv2.imshow(imageName+'.jpg', im)
#     #cv2.waitKey(0)
#     cv2.imwrite('../../../dataset/object_detection/annotated_images/'+image_name,im)
#     print('Image %s created successfully!' % image_name)